In [1]:
# Downloading the dataset
! python -m wget "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip"


Saved under rps.zip


In [2]:
import zipfile
with zipfile.ZipFile('rps.zip', 'r') as zip_ref:
    zip_ref.extractall()


In [1]:
# Downloading the weights
import urllib

weights = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

weights_file = 'inception_v3.h5'
urllib.request.urlretrieve(weights, weights_file)

URLError: <urlopen error unknown url type: https>